In [ ]:
import os
import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
ROOT_FOLDER = os.path.join(os.getcwd(), "plot_data")

# Robustness

In [ ]:
FILENAME = "robustness_350_class_all_small.csv"
FOLDER_PATH = os.path.join(ROOT_FOLDER, "robustness_data")
CSV_PATH = os.path.join(FOLDER_PATH, FILENAME)

df = pd.read_csv(CSV_PATH)
total_shift = np.add(np.abs(df["Shift_X"]), np.abs(df["Shift_Y"]))

In [ ]:
max_row = df.iloc[df['mIoU'].idxmax()]
hline_str = f"mIoU: {max_row['mIoU']}, Shift: {max_row['Shift_X']},{max_row['Shift_Y']}"
print(f"Best record: {max_row}")

min_iou = df['mIoU'].min()
max_iou = df['mIoU'].max()

In [ ]:
fig = px.scatter_3d(df, x='Angle', y=total_shift, z='mIoU',
              color='mIoU', width=1200, height=800, labels={"y":"Total Shift", "Angle": "Angle (rad)"}, range_color=[min_iou, max_iou])

fig.update_layout(title="Augmentation Robustness - All classes")
fig.update_xaxes(tickvals=df["Angle"])
fig.update_coloraxes(cmin=min_iou, cmax=max_iou)
fig.show()

In [ ]:
fig = px.scatter(df, x="Angle", y="mIoU", color=total_shift,
                 width=1200, height=600, labels={"color":"Total Shift", "Angle": "Angle (rad)"})
fig.update_xaxes(tickvals=df["Angle"])
fig.update_layout(title="Augmentation Robustness - All classes")
fig.show()

# Threshold Plots

In [ ]:
FILENAME = "th_argmax_500.csv"
FOLDER_PATH = os.path.join(ROOT_FOLDER, "threshold_data")
CSV_PATH = os.path.join(FOLDER_PATH, FILENAME)

df = pd.read_csv(CSV_PATH)
th_val_perc = df["Th_Value"] * 100

In [ ]:
max_row = df.iloc[df['IoU'].idxmax()]
hline_str = f"IoU: {np.round(max_row['IoU'], 3)}, Th_Values: {max_row['Th_Value']}"
print(f"Best record: {max_row}")

In [ ]:
fig = px.line(df, x=th_val_perc, y="IoU", width=1200, height=600, markers=True, labels={"x": "Th. Value (%)"})
fig.update_xaxes(tickvals=th_val_perc)
fig.add_trace(go.Scatter(x=[max_row['Th_Value'] * 100], y=[max_row['IoU']], mode = 'markers',
                         marker_symbol = 'star',
                         marker_size = 15,
                         showlegend=False))
# fig.update_layout(title="Best threshold value for Slice Var mode")
fig.show()

# Sweep plot

In [ ]:
FILENAME = "argmax_no_norm_l1.csv"
FOLDER_PATH = os.path.join(ROOT_FOLDER, "sweep_data")
CSV_PATH = os.path.join(FOLDER_PATH, FILENAME)

df = pd.read_csv(CSV_PATH)
avg_iou_single = df["aug_iou_single"]

In [ ]:
max_row = df.iloc[df['aug_iou_single'].idxmax()]
print(max_row)

In [ ]:
fig = px.scatter_3d(df, x='lambda_L1', y='lambda_L2', z='lambda_tv',
              color='aug_iou_single', width=1200, height=800,
              labels={"lambda_L1": "\u03B3 (L1)", "lambda_L2": "\u03BC (L2)", "lambda_tv": "\u03BB (TV)", "aug_iou_single": "Avg. IoU"})
# fig.update_coloraxes(cmin=avg_iou_single.min(), cmax=avg_iou_single.max(), colorbar=dict(tick0=avg_iou_single.min(), dtick=0.001))
fig.show()

In [ ]:
fig = px.scatter(df, x="lambda_L1", y="aug_iou_single", color="aug_iou_single",
                 width=1200, height=600, labels={"lambda_L1": "\u03B3 (L1)", "aug_iou_single": "Avg. IoU"})
fig.update_coloraxes(showscale=False)
fig.show()

In [ ]:
FILENAME = "slice_var_no_norm_no_l1.csv"
FOLDER_PATH = os.path.join(ROOT_FOLDER, "sweep_data")
CSV_PATH = os.path.join(FOLDER_PATH, FILENAME)

df = pd.read_csv(CSV_PATH)
avg_iou_single= df["aug_iou_single"]

In [ ]:
fig = px.scatter(df, x="lambda_L2", y="lambda_tv", color="aug_iou_single",
                 width=1200, height=600, labels={})
fig.update_layout()
fig.show()

# Final Evaluations

In [132]:
FILENAME = "slice_max_validation_final.csv"
CSV_PATH = os.path.join(ROOT_FOLDER, FILENAME)

df = pd.read_csv(CSV_PATH)

In [133]:
# df = df[::-1]
# index = np.arange(1, 21)
name = df["Name"]
aug_single = df["aug_iou_single"]
standard_single = df["standard_iou_single"]
mean_iou = df["mean_iou"]
max_iou = df["max_iou"]

aug_single_avg = aug_single.mean()
std_single_avg = standard_single.mean()

fig = go.Figure()
fig.add_trace(go.Bar(
    x=name,
    y=aug_single,
    name="ASR",
    marker_color='indianred',
    text=round(aug_single, 2),
    textposition='auto'
))
fig.add_trace(go.Bar(
    x=name,
    y=standard_single,
    name="Standard",
    marker_color='lightsalmon',
    text=round(standard_single, 2),
    textposition='auto'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis = {'type' : 'category'}, xaxis_title="Class ID", yaxis_title="IoU", bargap=0.05, width=1600)
# fig.add_hline(y=std_single_avg, annotation_text="Avg. Standard IoU", line_dash="dot")
# fig.add_hline(y=aug_single_avg, annotation_text="Avg. ASR IoU", line_dash="dot")
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()

In [134]:
aug_single_avg = aug_single.mean()
std_single_avg = standard_single.mean()

In [135]:
diff = aug_single - standard_single
avg = np.mean(diff)

print(avg * 100)

-19.017489348788292


In [136]:
diff_mean = mean_iou - standard_single
print(np.mean(diff_mean) * 100)

-19.295992468865567


In [137]:
diff_max = max_iou - standard_single
print(np.mean(diff_max) * 100)

-21.694074686095295


# Random Tests

In [ ]:
FILENAME = "argmax_500_sweep_all.csv"
CSV_PATH = os.path.join(ROOT_FOLDER, FILENAME)

df = pd.read_csv(CSV_PATH)

In [ ]:
dropout_perc = df["copy_dropout"] * 100

In [ ]:
fig = px.scatter(df, x=dropout_perc, y="avg_aug_SR_iou", color="use_BTV",                
                 width=1200, height=600, labels={"x":"Copy dropout (%)", "avg_aug_SR_iou":"avg. mIoU", "use_BTV":"BTV"})
fig.update_layout(title="Sample tuner runs for copy dropout and BTV - Class 8 + background")
fig.show()